In [1]:
import pandas as pd
import numpy as np
import re
import os
import string
import fn_module
from fn_module import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = fn_module.read_file("CSE508_Winter2023_Dataset_Processed_new")
print("File count = ",len(data))

File count =  1400


In [3]:
keys = list(data.keys())

In [4]:
s = ""
df = []
for key in data:
    df.append(data[key])


In [5]:
from collections import defaultdict

def create_inverted_index(documents):
    inverted_index = defaultdict(list)
    
    for doc_id, document in enumerate(documents):
        for word in document.split():
            inverted_index[word].append(doc_id)
    
    return inverted_index

In [6]:
inverted_index = create_inverted_index(df)

In [7]:

import pickle
from collections import defaultdict
def save_inverted_index(inverted_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(inverted_index, f)

def load_inverted_index(file_path):
    with open(file_path, 'rb') as f:
        inverted_index = pickle.load(f)
    
    return inverted_index

In [8]:
file_path = 'inverted_index.pickle'
save_inverted_index(inverted_index, file_path)

inverted_index = load_inverted_index(file_path)

In [9]:
def get_list(inverted_index , word):
    return inverted_index[word]

In [10]:
def and_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
        else:
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
    return final_set, count_comparisons

In [11]:
def or_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            final_set.add(s_2[ptr_2])
            ptr_2 = ptr_2 + 1
        else:
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
        
    while(ptr_1 < len_1):
        final_set.add(s_1[ptr_1])
        ptr_1 += 1
        
    while(ptr_2 < len_2):
        final_set.add(s_2[ptr_2])
        ptr_2 += 1
        
    result = sorted(final_set)
        
    return result, count_comparisons

In [12]:
def ornot_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_1 = ptr_1 + 1
            ptr_2 = ptr_2 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
        else:
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
        
    while(ptr_1 < len_1):
        final_set.add(s_1[ptr_1])
        ptr_1 += 1
        
        
    result = sorted(final_set)
        
    return result, count_comparisons

In [13]:
def andnot_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_1 = ptr_1 + 1
            ptr_2 = ptr_2 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            final_set.add(s_2[ptr_2])
            ptr_2 = ptr_2 + 1
        else:
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
    return final_set, count_comparisons

In [14]:
def retrieve_documents(output_sequence, operations , inverted_index, log = False):
    count = 0
    l1 = get_list(inverted_index,output_sequence[0])
    if log:
        print("L1 = ",l1)
    for i in range(len(operations)):
        l2 = get_list(inverted_index,output_sequence[i+1])
        if log:
            print("L2 = ",l2)
        op = []
        if(operations[i] == 'AND'):
            op, count_temp = and_operation(l1,l2)
        elif(operations[i] == 'AND NOT'):
            op, count_temp = andnot_operation(l1,l2)
        elif(operations[i] == 'OR'):
            op, count_temp = or_operation(l1,l2)
        elif(operations[i] == 'OR NOT'):
            op, count_temp = ornot_operation(l1,l2)
        if log:
            print("OPERATOR = ",operations[i])
            print("OUTPUT AFTER OPERTION",op)
            print("-"*100)
        
        l1 = op
        count+=count_temp

    return l1, count

In [20]:
t = int(input())

while(t > 0):
    t = t-1
    sequence  = input()
    oper = input()

    output_sequence = fn_module.preprocess(sequence)
 
    operations = oper.split(',')
    print("Processed sequence :",output_sequence)
    query = output_sequence[0]+' '
    i = 0
    while( i < np.size(operations)):
        query = query + operations[i] + ' ' + output_sequence[i+1] + ' '
        i= i+1
    print("Final Query :",query)

    # num_documents, operation_count = retrieve_documents(output_sequence, operations,inverted_index,True)
    num_documents, operation_count = retrieve_documents(output_sequence, operations,inverted_index)
    
    print("Number of documents retrieved for query","'"+query+"'","is :",len(num_documents))
    print("Names of the documents retrieved for","'"+query+"'","is :")
    
    if len(num_documents)==0:
        print("NO DOCUMENT FOUND!!")
    else:
        doc_lst = []
        for val in num_documents:
            doc_lst.append(keys[val])
        print(doc_lst)

    print("Number of comparisons required for","'"+query+"'","is :",operation_count)
    print("-"*150)

    
        

Processed sequence : ['car', 'bag', 'canister']
Final Query : car OR bag  AND NOT canister 
['car', 'bag', 'canister']
Number of documents retrieved for query 'car OR bag  AND NOT canister ' is : 0
Names of the documents retrieved for 'car OR bag  AND NOT canister ' is :
NO DOCUMENT FOUND!!
Number of comparisons required for 'car OR bag  AND NOT canister ' is : 0
------------------------------------------------------------------------------------------------------------------------------------------------------
Processed sequence : ['coffee', 'brewing', 'techniques', 'cookbook']
Final Query : coffee AND brewing  OR NOT techniques  OR cookbook 
['coffee', 'brewing', 'techniques', 'cookbook']
Number of documents retrieved for query 'coffee AND brewing  OR NOT techniques  OR cookbook ' is : 0
Names of the documents retrieved for 'coffee AND brewing  OR NOT techniques  OR cookbook ' is :
NO DOCUMENT FOUND!!
Number of comparisons required for 'coffee AND brewing  OR NOT techniques  OR cookb

In [16]:
print(inverted_index["canister"])

[]


In [17]:
len(inverted_index.keys())

8969